<a href="https://colab.research.google.com/github/anitarestrepo16/machine-learning-class-2023/blob/main/MiniProject3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import seaborn as sns
sns.set()
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings("ignore")
import math
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV

In [3]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [4]:
covid = pd.read_csv('/drive/My Drive/Colab Notebooks/Data-Covid002.csv', engine='python',encoding='latin1')
covid.head()

,county,state,fips,cases,deaths,population,casespc,deathspc,cty,county_name,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
0,Autauga,Alabama,1001,4434,152,918492,482.74780,16.548864,1001,Autauga,...,0.011183,0.0,306.02344,96.055420,288.08508,85.651848,11.712587,7893,859.29999,0
1,Baldwin,Alabama,1003,10465,278,3102984,337.25601,8.959118,1003,Baldwin,...,0.011756,0.0,305.51663,97.971542,290.20886,89.730972,10.077723,30292,976.20001,0
2,Barbour,Alabama,1005,3157,33,499262,632.33331,6.609756,1005,Barbour,...,0.012059,0.0,306.06226,97.371674,289.24210,88.633575,10.981967,5197,1040.90000,0
3,Bibb,Alabama,1007,2291,24,397470,576.39569,6.038192,1007,Bibb,...,0.008007,0.0,305.98218,96.293076,287.36282,86.485870,11.998714,4089,1028.80000,0
4,Blount,Alabama,1009,2082,15,997531,208.71532,1.503713,1009,Blount,...,0.007012,0.0,305.17886,94.630951,285.56567,85.449142,11.793022,9912,993.70001,0


In [5]:
covid.drop(['fips', 'casespc', 'cases', 'deaths', 'population', 'cty', 'county_name', 'cty_pop2000', 'cz', 'cz_name',
       'cz_pop2000', 'statename', 'state_id', 'stateabbrv', 'csa', 'csa_name',
       'cbsa', 'cbsa_name',], axis = 1, inplace=True)
covid.head()

,county,state,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
0,Autauga,Alabama,16.548864,1,0.333333,0.238095,0.208333,0.133333,0.375000,0.238095,...,0.011183,0.0,306.02344,96.055420,288.08508,85.651848,11.712587,7893,859.29999,0
1,Baldwin,Alabama,8.959118,1,0.268097,0.233503,0.167464,0.176991,0.298050,0.262467,...,0.011756,0.0,305.51663,97.971542,290.20886,89.730972,10.077723,30292,976.20001,0
2,Barbour,Alabama,6.609756,0,0.228571,0.250000,0.181818,0.111111,0.294118,0.571429,...,0.012059,0.0,306.06226,97.371674,289.24210,88.633575,10.981967,5197,1040.90000,0
3,Bibb,Alabama,6.038192,1,0.244444,0.280000,0.181818,0.150000,0.466667,0.375000,...,0.008007,0.0,305.98218,96.293076,287.36282,86.485870,11.998714,4089,1028.80000,0
4,Blount,Alabama,1.503713,1,0.304348,0.260870,0.352941,0.166667,0.347826,0.318182,...,0.007012,0.0,305.17886,94.630951,285.56567,85.449142,11.793022,9912,993.70001,0


In [13]:
na_vars = covid.isnull().sum()
na_vars[na_vars != 0]

reimb_penroll_adj10           4
mort_30day_hosp_z             1
adjmortmeas_amiall30day       1
med_prev_qual_z              95
primcarevis_10                9
diab_hemotest_10             38
diab_eyeexam_10              53
diab_lipids_10               50
mammogram_10                 78
amb_disch_per1000_10         65
gini99                       99
inc_share_1perc              99
frac_middleclass              1
rel_tot                       1
pop_d_2000_1980               7
lf_d_2000_1980                7
mig_inflow                   90
mig_outflow                  90
ccd_exp_tot                  27
ccd_pup_tch_ratio           266
score_r                      38
dropout_r                   628
tuition                    1816
gradrate_r                 1808
e_rank_b                    259
crime_total                 175
tax_st_diff_top20             1
dtype: int64

In [14]:
covid.dropna(inplace=True)
na_vars = covid.isnull().sum()
na_vars[na_vars != 0]

Series([], dtype: int64)

In [22]:
covid.state.unique()

array(['Alabama', 'Arizona', 'Arkansas', 'Connecticut', 'Delaware',
       'Georgia', 'Idaho', 'Illinois', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Texas', 'Utah', 'Vermont',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [23]:
pd.get_dummies(covid, columns = ["state"])

,county,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,bmi_obese_q3,...,state_Pennsylvania,state_Rhode Island,state_South Carolina,state_South Dakota,state_Texas,state_Utah,state_Vermont,state_West Virginia,state_Wisconsin,state_Wyoming
1,Baldwin,8.959118,1,0.268097,0.233503,0.167464,0.176991,0.298050,0.262467,0.193237,...,0,0,0,0,0,0,0,0,0,0
7,Calhoun,6.413799,1,0.294118,0.240356,0.170886,0.182482,0.250000,0.317073,0.266234,...,0,0,0,0,0,0,0,0,0,0
15,Coffee,2.684003,1,0.196078,0.289474,0.100000,0.171429,0.369565,0.277778,0.366667,...,0,0,0,0,0,0,0,0,0,0
16,Colbert,9.585782,1,0.354839,0.250000,0.000000,0.190476,0.409836,0.463415,0.416667,...,0,0,0,0,0,0,0,0,0,0
17,Conecuh,4.582779,0,0.272727,0.444444,0.000000,0.000000,0.300000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3094,Laramie,3.802303,1,0.340897,0.244410,0.162342,0.135569,0.227248,0.212970,0.209125,...,0,0,0,0,0,0,0,0,0,1
3096,Natrona,0.302376,1,0.353198,0.257286,0.207684,0.135472,0.261434,0.219814,0.216702,...,0,0,0,0,0,0,0,0,0,1
3098,Park,0.000000,0,0.235222,0.182682,0.136038,0.075881,0.212198,0.176216,0.170316,...,0,0,0,0,0,0,0,0,0,1
3100,Sheridan,0.000000,0,0.320755,0.178330,0.189573,0.107623,0.211679,0.202546,0.214976,...,0,0,0,0,0,0,0,0,0,1


In [26]:
train, test = train_test_split(covid, test_size = 0.2, random_state = 25)
display(train.head())
display(test.head())

,county,state,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
497,Troup,Georgia,26.594490,1,0.090909,0.062500,0.000000,0.071429,0.500000,0.250000,...,0.019655,0.00,305.61523,95.940887,287.54749,87.799347,11.622005,11720,1005.40000,0
1583,Hill,Montana,0.000000,0,0.363636,0.215116,0.201705,0.153846,0.276074,0.270833,...,0.019672,0.00,300.47922,83.122429,273.12369,90.844872,3.380106,2669,911.40002,0
1940,Surry,North Carolina,2.524553,1,0.370656,0.295455,0.142857,0.088235,0.307054,0.272189,...,0.015248,1.00,302.95798,93.797501,282.86664,77.992798,10.357016,14841,1135.40000,0
1985,Mountrail,North Dakota,0.000000,0,0.388889,0.312500,0.200000,0.000000,0.555556,0.533333,...,0.017019,3.44,299.31897,89.665260,268.14575,91.298195,4.423579,1516,1087.00000,0
507,Wayne,Georgia,0.000000,1,0.211538,0.250000,0.217391,0.100000,0.400000,0.428571,...,0.026402,0.00,306.64200,98.347206,291.20850,90.117737,10.349192,5184,995.79999,0


,county,state,deathspc,intersects_msa,cur_smoke_q1,cur_smoke_q2,cur_smoke_q3,cur_smoke_q4,bmi_obese_q1,bmi_obese_q2,...,taxrate,tax_st_diff_top20,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax,pm25,bmdeaths,bmcruderate,pm25_mia
1158,Allegany,Maryland,33.794163,1,0.272849,0.220513,0.140845,0.144186,0.272981,0.224599,...,0.018325,1.0,301.25504,91.034729,277.47534,85.105438,10.858380,16450,1232.60000,0
365,Ben Hill,Georgia,5.738515,0,0.409091,0.263158,1.000000,0.166667,0.409091,0.470588,...,0.021014,0.0,306.56223,96.990685,289.94113,87.198586,10.855447,3761,1199.00000,0
462,Muscogee,Georgia,15.537071,1,0.261146,0.209354,0.166667,0.097222,0.329646,0.245496,...,0.019444,0.0,306.17218,94.239967,288.63913,84.645096,12.671142,32654,951.90002,0
1142,Androscoggin,Maine,4.373542,1,0.300921,0.234973,0.167038,0.082645,0.281481,0.263873,...,0.024858,0.0,298.23334,88.627174,273.00107,83.580750,6.017636,18829,980.40002,0
1848,Ulster,New York,36.196602,1,0.339623,0.210526,0.214286,0.156463,0.293333,0.224242,...,0.042328,0.0,299.15216,89.545975,274.80234,83.697891,8.673356,28527,875.79999,0


# Q6

In [35]:
X = train.drop(["deathspc", "county", "state"], axis = 1)
y = train['deathspc']
mod = LinearRegression().fit(X, y)

Training MSE:  

In [39]:
y_pred = mod.predict(train.drop(["deathspc", "county", "state"], axis = 1))
mean_squared_error(train['deathspc'], y_pred)

1468.5822663864965

Test MSE:  

In [40]:
y_pred = mod.predict(test.drop(["deathspc", "county", "state"], axis = 1))
mean_squared_error(test['deathspc'], y_pred)

4080.383184357558

# Q7

## Lasso

In [43]:
alphas = np.linspace(.01, 100, 100)
lasso_cv = make_pipeline(StandardScaler(), LassoCV(alphas=alphas, cv=10, random_state=1)).fit(X, y)

In [48]:
# pull the model results from the pipeline object
lasso = lasso_cv[-1]
lasso

LassoCV(alphas=array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+0...
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 8.283e+01, 8.384e+01,
       8.485e+01, 8.586e+01, 8.687e+01, 8.788e+01, 8.889e+01, 8.990e+01,
       9.091e+01, 9.192e+01, 9.293e+01, 9.394e+01, 9.495e+01, 9.596e+01,
       9.697e+01, 9.798e+01, 9.899e+01, 1.000e+02]),
        cv=10, random_state=1)

In [54]:
lasso.alpha_

1.02

## Ridge Regression